# ****IMPORTANT NOTE:**

This notebook is for VIEW ONLY. To test and run the notebook, please download, upload and run the zeppelin notebook on Peel: [Cleaning_Dataset_peel_Completeness.zpln](https://github.com/qyc206/evq_big_data_project/blob/main/notebooks/part2/Cleaning_Dataset_peel_Completeness.zpln).

# Cleaning

This notebook goes over the cleaning process and results of cleaning the dataset based on the problems observed in the Profiling_The_Dataset.ipynb notebook. Some methods are further modified to have mroe general uses for datasets that are not the original. However, to show our algorithms, we use the original dataset as an example.

Just like the notebook for profiling, this notebook will also be broken down into the following sections: Uniformity, Accuracy, Inconsistency, Completeness, and Outlier. Each section will show the solution that we have come up with to solve the problem posed in the corresponding section in the profiling notebook.

## Upload the dataset to Peel cluster & Define dataset path

Before continuing, make sure your dataset is available on Peel HDFS. If your dataset is on your local machine, you can copy them to the login node of the cluster and move them to your user directory in the HDFS using the following commands:

```
# Copy file from local machine to login node of the cluster
mylaptop$ scp -r [FILENAME] <net_id>@peel.hpc.nyu.edu:~/

# Move file from cluster login node to your user directory in HDFS 
# (your file will be in the path "/user/[netid]/[FILENAME]")
hfs -put [FILENAME]
```

Make sure you can locate your dataset before continuing onwards.

In [ ]:
%pyspark
# Define path to dataset on Peel HDFS (NOTE: replace file name with your own if different)
dataset_path = "/user/CS-GY-6513/project_data/data-cityofnewyork-us.erm2-nwe9.csv"

## Set up Spark Session

Now that the dataset is uploaded and the path is defined, we need to set up pyspark to begin profiling and exploring our dataset. 

If this notebook is run in an environment where pyspark is not yet installed, please add a new cell BEFORE the next cell and run the following command:

```
# Run this command if pyspark is not already installed
%pip install pyspark
```

In [ ]:
%pyspark

# Set up pyspark session
from pyspark.sql import SparkSession

spark = SparkSession \
            .builder \
            .appName("Python Spark SQL basic example") \
            .config("spark.some.config.option", "some-value") \
            .config("spark.executor.memory", "50g") \
            .config("spark.driver.memory", "50g") \
            .getOrCreate()

## Load dataset using spark

Run the following lines to load the dataset using spark and test to make sure that dataset is properly loaded.

In [ ]:
%pyspark

# Load dataset
df = spark.read.format('csv').options(header='true',inferschema='true').load(dataset_path)
# (Note: change "311_service_report" to a name that better suits your dataset, if different)
df.createOrReplaceTempView("311_service_report") 

Notice in the result of running the above cell that most items in the schema is of type string, even if it is not the expected type. To modify the dataset such that the types of each column is what we would expect, we perform type casting for each column that should not be type string.

**NOTE: the following cell is specific for the 311 service report dataset; make sure to modify the following cell to include type casting that is necessary to your dataset, if different

In [ ]:
%pyspark

from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import to_timestamp

# Type casting to expected types
df = df.withColumn("Unique Key",df["Unique Key"].cast(IntegerType()))
df = df.withColumn("Due Date",to_timestamp(df["Due Date"],"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Created Date", to_timestamp(df["Created Date"],"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Closed Date",to_timestamp(df["Closed Date"],"MM/dd/yyyy hh:mm:ss a"))
df = df.withColumn("Incident Zip",df["Incident Zip"].cast(IntegerType()))
df = df.withColumn("BBL",df["BBL"].cast(IntegerType()))
df = df.withColumn("X Coordinate (State Plane)",df["X Coordinate (State Plane)"].cast(IntegerType()))
df = df.withColumn("Y Coordinate (State Plane)",df["Y Coordinate (State Plane)"].cast(IntegerType()))
df = df.withColumn("Resolution Action Updated Date",to_timestamp(df["Resolution Action Updated Date"],"MM/dd/yyyy hh:mm:ss a"))


# (Note: change "311_service_report" to a name that better suits your dataset, if different)
df.createOrReplaceTempView("311_service_report")

df.printSchema()

## IV. Completeness

During profiling, we observed that several values in the City column are null (i.e. missing). One simple way to try and fix the city nulls is by using the "Borough" Column because the data set seems to use Borough names for city names. Therefore, we simply replace the null values that have a non null value for its Borough with the Borough name.

Manhattan is a slightly special case because New York = Manhattan in the city column. For this case we have to change the value of "MANHATTAN" in Borough to New York.

Any null value in "City" that also has a null "Borough" value will be marked as Unspecified.

To make this function more applicable to general uses, there a flag for whether you will be able to apply one column to complete the other, or if you are forced to only fill in “unspecified” for one column. If there are any two columns that have a similar relationship, the function could be tweaked (ie get rid of the Manhattan to New York cases) to fit the function better

In [ ]:
%pyspark
# Replace null values with "unspecified"
from pyspark.sql.functions import when, desc, initcap

def completeColumn(datframe, targetCol,crossCheckCol,crossColFlag):
    if(crossColFlag == True):
        #if we want to cross check our data, we will cross check it here
        datframe = datframe.withColumn(targetCol,\
              when(datframe[targetCol].isNull() & (datframe[crossCheckCol] == "MANHATTAN"), "New York").when(datframe[targetCol].isNull() &  datframe[crossCheckCol].isNotNull() & (datframe[crossCheckCol] != "MANHHATAN"), initcap(df[crossCheckCol])).when(datframe[targetCol].isNull() &  datframe[crossCheckCol].isNull(), "Unspecified").otherwise(datframe[targetCol]))
    else:
        #Mark the row as unspecified
        datframe = datframe.withColumn(targetCol, when(datframe[targetCol].isNull(), "Unspecified").otherwi(datframe[targetCol]))
    
    return datframe

In [ ]:
%pyspark

# Replace null values with "unspecified"

df = completeColumn(df, "City","Borough", True)

In [ ]:
%pyspark

# Re-display number of values in City column
df.groupBy('City').count().orderBy(desc("count")).show(20, False)

In [ ]:
%pyspark
# Count how many nulls left
df.filter(df["City"].isNull()).count()